# PCOSense: An Explainable & Patient-Centric PCOS Risk Intelligence System

**Author:** Raksha Mishra  
**Project:** CycleSense / PCOSense  
**Domain:** Women’s Health · Machine Learning · Tableau Analytics  

---

## Why This Study Exists

I was diagnosed with PCOS at the age of **15**.

What followed were years of acne flare-ups, hirsutism, irregular menstrual cycles,
weight fluctuations, and eventually thyroid imbalance. Treatment plans changed,
medications rotated, and explanations often remained incomplete.

Most clinical tools reduce PCOS to a *binary diagnosis*.

This project challenges that.

**PCOSense** is built to:
- Quantify PCOS risk on a **spectrum**
- Combine **clinical data + lived symptoms**
- Provide **explainable insights**, not black-box predictions
- Act as a **decision-support system**, not a diagnosis

> This notebook documents the full research, modeling, and interpretability pipeline.
